<a href="https://www.kaggle.com/code/shubkumarop/codebreak-1-0-nlp-for-beginners?scriptVersionId=132563538" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/codebreak-10-nlp-with-disaster-tweets/sample_submission.csv
/kaggle/input/codebreak-10-nlp-with-disaster-tweets/train.csv
/kaggle/input/codebreak-10-nlp-with-disaster-tweets/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/codebreak-10-nlp-with-disaster-tweets/train.csv")
test_df = pd.read_csv("/kaggle/input/codebreak-10-nlp-with-disaster-tweets/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                            train_df["target"].to_numpy(),
                                                                            test_size=0.1, 
                                                                            random_state=42) 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
train_sentences[:10], train_labels[:10]

(array(["'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j",
        'w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
        "When I was cooking earlier I got electrocuted some crucial ?????? now I'm psychic lol",
        "I'm On Fire.  http://t.co/WATsmxYTVa",
        "More than 40 families affected by the fatal outbreak of Legionnaires' disease in Edinburgh are to sue two comp... http://t.co/vsoXioOy78",
        'Why do u ruin everything?  @9tarbox u ruined the sour cream and u put a brick of cheese in the freezer..dummy',
        'http://t.co/c1H7JECFrV @RoyalCarribean do your passengers know about the mass murder that takes place in the #FaroeIslands every year?',
        "WACKOES like #MicheleBachman predict the WORLD will SOON be OBLITERATED by a burning firey INFERNO but can't accept #GlobalWarming!! HELLO!!",
        '@Reuters people like you should be charged after the inevitable rioting for contributing to

In [6]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [7]:
max_vocab_length = 10000 
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [8]:
text_vectorizer.adapt(train_sentences)

In [9]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=max_length, 
                             name="embedding_1") 

embedding

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


model = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [11]:
score = model.score(val_sentences, val_labels)

In [12]:
preds = model.predict(val_sentences)
preds[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [13]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_result(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision, model_recall, model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average="weighted")
    model_result = {"accuracy":model_accuracy,
                    "precision":model_precision,
                    "recall":model_recall,
                    "f1":model_f1}
    return model_result

In [14]:
results = calculate_result(y_true=val_labels,
                                     y_pred=preds)
results

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}

In [15]:
test_df = test_df['text']

In [16]:
Submission = pd.read_csv("/kaggle/input/codebreak-10-nlp-with-disaster-tweets/sample_submission.csv")
Submission["target"] = model.predict(test_df)

In [17]:
Submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [18]:
Submission.to_csv('submission.csv', index=False)